## Part 1: Preprocessing

In [38]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import concatenate


#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [39]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [40]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [41]:
# Create a list of at least 10 column names to use as X data
feature_columns = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked',
]


# Create X_df using your selected columns
X_df = attrition_df[feature_columns]

# Show the data types for X_df
data_types = X_df.dtypes
print(data_types)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [42]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df['Attrition'], test_size=0.2, random_state=42)


In [43]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
encoder = OneHotEncoder(sparse_output=False, drop='first')

X_encoded = encoder.fit_transform(X_df[feature_columns])

X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(feature_columns))

X_df_numeric = X_df.drop(columns=feature_columns)
X_final = pd.concat([X_df_numeric, X_encoded_df], axis=1)

print(X_final.head())
print(X_final.dtypes)


   Education_2  Education_3  Education_4  Education_5  Age_19  Age_20  Age_21  \
0          1.0          0.0          0.0          0.0     0.0     0.0     0.0   
1          0.0          0.0          0.0          0.0     0.0     0.0     0.0   
2          1.0          0.0          0.0          0.0     0.0     0.0     0.0   
3          0.0          0.0          1.0          0.0     0.0     0.0     0.0   
4          0.0          0.0          0.0          0.0     0.0     0.0     0.0   

   Age_22  Age_23  Age_24  ...  YearsSinceLastPromotion_15  \
0     0.0     0.0     0.0  ...                         0.0   
1     0.0     0.0     0.0  ...                         0.0   
2     0.0     0.0     0.0  ...                         0.0   
3     0.0     0.0     0.0  ...                         0.0   
4     0.0     0.0     0.0  ...                         0.0   

   NumCompaniesWorked_1  NumCompaniesWorked_2  NumCompaniesWorked_3  \
0                   0.0                   0.0                   0.0  

In [44]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_final.loc[X_train.index])

# Scale the training and testing data
X_test_scaled = scaler.transform(X_final.loc[X_test.index])



In [45]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
dept_encoder.fit(y_df.loc[X_train.index, ['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_train_encoded = dept_encoder.transform(y_df.loc[X_train.index, ['Department']])
dept_test_encoded = dept_encoder.transform(y_df.loc[X_test.index, ['Department']])

print("Encoded Department for Training Set:")
print(dept_train_encoded)

print("\nEncoded Department for Testing Set:")
print(dept_test_encoded)


Encoded Department for Training Set:
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]

Encoded Department for Testing Set:
[[0. 1.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [

In [46]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
attrition_encoder.fit(y_df.loc[X_train.index, ['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_df.loc[X_train.index, ['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_df.loc[X_test.index, ['Attrition']])

print("Encoded Attrition for Training Set:")
print(attrition_train_encoded)

print("\nEncoded Attrition for Testing Set:")
print(attrition_test_encoded)

Encoded Attrition for Training Set:
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]

Encoded Attrition for Testing Set:
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.

## Create, Compile, and Train the Model

In [47]:
# Find the number of columns in the X training data
model = Sequential()
num_features = X_train_scaled.shape[1]

# Create the input layer
model.add(layers.InputLayer(input_shape=(num_features,)))

# Create at least two shared layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [48]:
# Create a branch for Department
# with a hidden layer and an output layer
main_input = Input(shape=(num_features,), name='main_input')
dept_input = Input(shape=(dept_train_encoded.shape[1],), name='department_input')

# Create the hidden layer
dept_hidden = layers.Dense(32, activation='relu')(dept_input)

# Create the output layer
dept_output = layers.Dense(1, activation='sigmoid', name='department_output')(dept_hidden)


In [49]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_input = Input(shape=(attrition_train_encoded.shape[1],), name='attrition_input')

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(attrition_input)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [50]:
# Concatenate the outputs from all branches
merged_output = concatenate([dept_output, attrition_output])

# Add a final layer for prediction
final_output = layers.Dense(1, activation='sigmoid', name='final_output')(merged_output)


In [64]:
# Create the model
model = Model(inputs=[main_input, dept_input, attrition_input], outputs=final_output)

# Compile the model, change loss to 'binary_crossentropy'
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Corrected loss function
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ department_input          │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_input           │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 32)             │             96 │ department_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │             64 │ attrition_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 1)              │             33 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             33 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 2)              │              0 │ department_output[0][… │
│ (Concatenate)             │                        │                │ attrition_output[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ main_input (InputLayer)   │ (None, 144)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ final_output (Dense)      │ (None, 1)              │              3 │ concatenate_1[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 229 (916.00 B)

 Trainable params: 229 (916.00 B)

 Non-trainable params: 0 (0.00 B)

In [65]:
# Assuming y_train is a pandas Series or DataFrame
if isinstance(y_train, (pd.Series, pd.DataFrame)):
    # Check for and handle non-numeric values in y_train
    if y_train.dtype == object:  # If dtype is object, it likely contains strings
        print("Warning: y_train contains non-numeric values. Attempting to convert...")
        # Attempt to convert to numeric using a more robust method
        # Example: If values are 'Yes'/'No', map them to 1/0
        y_train = y_train.map({'Yes': 1, 'No': 0}).astype(int) # Replace 'Yes'/'No' with your actual values
    else:
        # Convert to integer for sparse_categorical_crossentropy if not already
        y_train = y_train.astype(int)

In [66]:
# Train the model
model.fit([X_train_scaled, dept_train_encoded, attrition_train_encoded],
          attrition_train_encoded,
          epochs=100,
          batch_size=32,
          validation_data=([X_test_scaled, dept_test_encoded, attrition_test_encoded], attrition_test_encoded))

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1563 - loss: 0.8208 - val_accuracy: 0.1327 - val_loss: 0.7994
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1591 - loss: 0.7828 - val_accuracy: 0.1327 - val_loss: 0.7645
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1620 - loss: 0.7503 - val_accuracy: 0.1327 - val_loss: 0.7348
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1852 - loss: 0.7192 - val_accuracy: 0.1327 - val_loss: 0.7090
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1872 - loss: 0.6971 - val_accuracy: 0.1327 - val_loss: 0.6869
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6743 - loss: 0.6793 - val_accuracy: 1.0000 - val_loss: 0.6673
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.6624 - val_accuracy: 1.0000 - val_loss: 0.6495
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9956 - loss: 0.6464 - val_accuracy: 0.8673 - v

In [69]:
# Evaluate the model with the testing data
test_loss, test_acc = model.evaluate([X_test_scaled, dept_test_encoded, attrition_test_encoded], attrition_test_encoded)

print('Test accuracy:', test_acc)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.1754 
Test accuracy: 1.0


In [70]:
# Predict outputs for the Department and Attrition branches
dept_predictions = model.predict([X_test_scaled, dept_test_encoded, attrition_test_encoded])
attrition_predictions = model.predict([X_test_scaled, dept_test_encoded, attrition_test_encoded])

# For binary classification, convert predictions to binary outcomes (0 or 1)
department_predictions_binary = (dept_predictions > 0.5).astype(int)
attrition_predictions_binary = (attrition_predictions > 0.5).astype(int)

# Calculate accuracies
dept_accuracy = np.mean(department_predictions_binary == dept_test_encoded)
attrition_accuracy = np.mean(attrition_predictions_binary == attrition_test_encoded)

# Print the accuracy for both department and attrition
print(f'Department Accuracy: {dept_accuracy:.4f}')
print(f'Attrition Accuracy: {attrition_accuracy:.4f}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Department Accuracy: 0.5153
Attrition Accuracy: 1.0000


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not be the best metric to use for this data! Classes are imbalanced.In cases of class imbalance, a high accuracy can be misleading, as the model might simply predict the majority class most of the time.

2. I chose the sigmoid activation function for the output layers because it is suitable for binary classification tasks. The sigmoid function outputs a value between 0 and 1, which can be interpreted as the probability of belonging to the positive class. This makes it easy to determine the output to classify the predictions effectively.

3. More Complex Models: Adding more layers or neurons might improve the
   model's ability to capture complex patterns in the data.
   
   Cross-Validation: Using k-fold cross-validation can provide a more reliable estimate of the model’s performance and help mitigate issues related to overfitting or selection bias.